### Imports

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau

from torch.utils.data import Dataset
from neuralnetworks import DynamicSeqRegNN, RegressionDataset

In [ ]:
torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())

True


### Data Preparation

In [ ]:
# Loading saved Preprocessed data
X_pca = pd.read_parquet("../Data/Processed/X_pca.parquet", engine="pyarrow")
y = pd.read_parquet("../Data/Processed/y.parquet", engine="pyarrow")

In [ ]:
# Splitting the PCA data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

In [ ]:
# Creating tensors for train and test split data
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.float32).view(-1, 1)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [ ]:
# Creating Dataset & DataLoader
batch_size = 120  
train_dataset = RegressionDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

### Sequential Model

In [ ]:
# Initialize Model
model = DynamicSeqRegNN(input_size=X_train_tensor.shape[1], hidden_sizes=[128, 64, 32])#, dropout=0.1)

# Defining training parameters
learning_rate = 0.1
num_epochs = int(4e3)

# Defining loss function
loss_fn = nn.MSELoss(reduction='sum') # Reduction tell how to aggregate the loss for multiple elements in a batch/input

# Selecting optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5) # Passing weights to the model to optimizer
 
# Adding a Learning Rate Scheduler (Reduces LR when loss stops improving)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

best_loss = float('inf')  # Store best loss
best_model = None
loss_rmse_df = pd.DataFrame(columns=['epoch', 'loss', 'rmse'])

# Training loop
for epoch in range(num_epochs):
    for batch_X, batch_y in train_loader:
        # Steps: Forward pass, calculate loss, zero gradients, backward pass, update weights and reduce lr if loss plateaus
        y_pred = model(batch_X)
        loss = loss_fn(y_pred, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step(loss)

        # Storing the best model in case we overstep
        if loss.item() < best_loss:
            best_loss = loss.item()
            best_model = model.state_dict()
        
    # Model evaluation and printing loss
    if epoch % 100 == 99:
        print(f'Epoch {epoch+1}, Loss {loss.item()}')
        model.eval()
        with torch.no_grad():
            y_pred = model(X_test_tensor)
        mse = loss_fn(y_pred, y_test_tensor)
        rmse = torch.sqrt(mse)
        logged_data = {'epoch': epoch+1, 'loss': loss.item(), 'rmse': rmse}
        loss_rmse_df.loc[len(loss_rmse_df)] = logged_data


Epoch 100, Loss 87477.140625
Epoch 200, Loss 87452.4453125
Epoch 300, Loss 87428.1796875


KeyboardInterrupt: 

In [ ]:
# px.line(loss_rmse_df, x='epoch', y= 'loss', title='Loss vs RMSE')
fig = px.line(loss_rmse_df, x='epoch', y='rmse', title='epoch vs RMSE of validation data')
fig 


In [ ]:
# Displaying Neural Network Model layers
for name, module in model.named_modules():
    print(f"{name}: {module}")

torch.save(model, "./saved_models/Neural Networks/NN_seq_model.pth")
torch.save(model.state_dict(), "./saved_models/Neural Networks/NN_seq_model_weights.pth")

: DynamicSeqRegNN(
  (activation): ReLU()
  (model): Sequential(
    (0): Linear(in_features=1, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=1, bias=True)
  )
)
activation: ReLU()
model: Sequential(
  (0): Linear(in_features=1, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=32, bias=True)
  (5): ReLU()
  (6): Linear(in_features=32, out_features=1, bias=True)
)
model.0: Linear(in_features=1, out_features=128, bias=True)
model.2: Linear(in_features=128, out_features=64, bias=True)
model.4: Linear(in_features=64, out_features=32, bias=True)
model.6: Linear(in_features=32, out_features=1, bias=True)


In [ ]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
mse = loss_fn(y_pred, y_test_tensor)
rmse = torch.sqrt(mse)
r2 = r2_score(y_test_tensor, y_pred.detach().numpy())
print(f'Root Mean Squared Error: {rmse}')
print(f'R-squared: {r2}')

Root Mean Squared Error: 4234.59326171875
R-squared: 0.04898494482040405


The model performace of the neural network is very poor compared to the rf and xgboost models